In [58]:
from gtfparse import read_gtf
import pandas as pd
import numpy as np
import pyfaidx

In [2]:
gtffile = "/Users/milessmith/workspace/ensembl/Homo_sapiens.GRCh38.94.gtf.gz"

In [59]:
fastafile = "/Users/milessmith/workspace/ensembl/Homo_sapiens.GRCh38.dna.primary_assembly.fa"

In [3]:
records = read_gtf(gtffile)

INFO:root:Extracted GTF attributes: ['gene_id', 'gene_version', 'gene_name', 'gene_source', 'gene_biotype', 'transcript_id', 'transcript_version', 'transcript_name', 'transcript_source', 'transcript_biotype', 'tag', 'transcript_support_level', 'exon_number', 'exon_id', 'exon_version', 'protein_id', 'protein_version', 'ccds_id']


In [12]:
only_starts = records.groupby('gene_name').apply(lambda x: x[pd.to_numeric(x['exon_number']) == 1])

In [33]:
only_starts['start'] = pd.to_numeric(only_starts['start'])

In [34]:
only_starts.head()

seqname          source      feature     start       end  \
gene_name                                                                    
A1BG      2638883      19          havana         exon  58347353  58347634   
          2638886      19  ensembl_havana         exon  58353404  58353499   
          2638887      19  ensembl_havana          CDS  58353404  58353437   
          2638888      19  ensembl_havana  start_codon  58353435  58353437   
          2638907      19          havana         exon  58353404  58353491   

                   score strand frame          gene_id gene_version  ...  \
gene_name                                                            ...   
A1BG      2638883    NaN      -     0  ENSG00000121410           11  ...   
          2638886    NaN      -     0  ENSG00000121410           11  ...   
          2638887    NaN      -     0  ENSG00000121410           11  ...   
          2638888    NaN      -     0  ENSG00000121410           11  ...   
          2638907    NaN      -     0  ENSG00000121410           11  ...   

                  transcript_source    transcript_biotype         tag  \
gene_name                                                               
A1BG      2638883            havana  processed_transcript               
          2638886    ensembl_havana        protein_coding  CCDS,basic   
          2638887    ensembl_havana        protein_coding  CCDS,basic   
          2638888    ensembl_havana        protein_coding  CCDS,basic   
          2638907            havana       retained_intron               

                  transcript_support_level exon_number          exon_id  \
gene_name                                                                 
A1BG      2638883                        1           1  ENSE00003033140   
          2638886                        1           1  ENSE00001247383   
          2638887                        1           1                    
          2638888                        1           1                    
          2638907                        2           1  ENSE00003032421   

                  exon_version       protein_id protein_version    ccds_id  
gene_name                                                                   
A1BG      2638883            1                                              
          2638886            2                                   CCDS12976  
          2638887               ENSP00000263100               2  CCDS12976  
          2638888                                                CCDS12976  
          2638907            1                                              

[5 rows x 26 columns]

In [42]:
only_starts = only_starts.reset_index(drop=True).groupby('gene_name').apply(lambda y: y.nsmallest(1,"start")).reset_index(drop=True)

In [43]:
len(only_starts)

57169

In [48]:
forward_starts = records[records['strand'] == "+"].\
    groupby('gene_name').\
    apply(lambda x: x[pd.to_numeric(x['exon_number']) == 1]).\
    reset_index(drop=True).\
    groupby('gene_name').\
    apply(lambda y: y.nsmallest(1, "start")).\
    reset_index(drop=True)

In [51]:
reverse_starts = records[records['strand'] == "-"].\
    groupby('gene_name').\
    apply(lambda x: x[pd.to_numeric(x['exon_number']) == 1]).\
    reset_index(drop=True).\
    groupby('gene_name').\
    apply(lambda y: y.nlargest(1, "start")).\
    reset_index(drop=True)

In [52]:
predicted_tss = pd.concat([forward_starts, reverse_starts])
predicted_tss = predicted_tss[['seqname', 'feature', 'start', 'end', 'strand', 'frame', 'gene_name']].drop_duplicates()

In [53]:
predicted_tss.head()

,seqname,feature,start,end,strand,frame,gene_name
0,19,exon,58347751,58347844,+,0,A1BG-AS1
1,12,exon,9065177,9065228,+,0,A2M-AS1
2,12,exon,8822472,8822713,+,0,A2ML1
3,12,exon,125065379,125065717,+,0,AACS
4,3,exon,151814037,151814300,+,0,AADAC


In [54]:
len(predicted_tss)

57264

In [60]:
sequences = pyfaidx.Fasta(fastafile)

In [67]:
feature_type = "gene"
boundary = 100
final_list = []
for rec in predicted_tss.itertuples():
    if rec.start != rec.end: # you'd be surprised
        if boundary != 0:
            try:
                seq = pyfaidx.Sequence(name=f"{rec.gene_name}_TSS"
                                            f"_{rec.strand}_{rec.start}_{rec.end}",
                                       seq=sequences[rec.seqname][rec.start-boundary:rec.start+boundary].seq)
                final_list.append(seq)
            except ValueError:
                print(f"problem with {rec.gene_name} {rec.start} {rec.end} {rec.seqname} {rec.strand}")

In [68]:
final_list

[>A1BG-AS1_TSS_+_58347751_58347844
 GGGCGGGTGGTCGGGCCAGGCCACGCCCCAGGCCACGCCCCAGGCCACACCCCAGGCCACACCCCAGGCCGCGCCCGCGCCTGCGCCTCAGCCCCGGCTTCATCTTCCCTGTCTTGTTCCTGGGCGCAGAGGGCTCCTCCGGCGCCGCCGAGCTGCGCCCGCTGCCTTCACACCCCGGACATCCGCGCCACACGTGAGTG,
 >A2M-AS1_TSS_+_9065177_9065228
 AGAACTTGCAGCGACGTCATCCCTGCCCCGGACACAGTCCCGGGCCCAGGGAGGACCTGGAACATTCCCTTCCTCCCCCCCCCCCCCCCCGCCCCCGCCCATCAGCCCAGCCTGCAAGGAGGCGCCACCGGACCTGCACACTCAGGAGCAGGTGAGCCAGGGACAAGCAGGAGCCCCGCCCCTTCCGAACTGGTGGAGCA,
 >A2ML1_TSS_+_8822472_8822713
 ATGAATACTCAGCTCTGGTTTTCCCCCAAAACTGGCTGAACCAGTATGGATGTTCCAGGCAGAAACAACTTTAAAGAAGTGAATAGTGGCCCTACCCTGGTCCACCACCTAACCTGGTGGTTACAGAAGGAGGAACACAGTCCTCCTGTGTTCCAGGTGCCAGGAGGTGACTGACTATAGCATCAGTCATCACTGCACAC,
 >AACS_TSS_+_125065379_125065717
 GAGCCGGCGGCTGCGAGCACTCCTTTGAGCGGCGCTGCACTCAGCGGAGGGTTACAGGCGGGAGGGCGGGGCCTGGAAGGGGGCGGGGATTCGAGGCGGGGCCTGCGGGGCGGGGCCTGGGCCAAGCGGCCCGCAGGAGGCGGCGGCGGCCGTTCAGTCCCTTGTTCCCCGCCGCCGCCGTCGCTGACCCAGCCCGCCAG,
 >AADAC_TSS_+_151814037_151814300
 TTTCCTATTACTCATTAAACCTT